# Fundamental Theorem of Simulation
The Fundamental Theorem of Simulation aka. Universality of the Uniform tells us that we can sample random-variables of any distribution, given its inverse CDF (or approximation) and a standard uniform rv. Basically $F^{-1}(U)=X$

## Sample binomial random-variable
Here we have to use an approximation of the inverse CDF.

In [2]:
import numpy as np
from scipy.special import comb
from scipy.stats import norm

In [3]:
def binom(n, k, p):
    return comb(n, k) * np.power(p, k) * np.power(1-p, n-k)
  
def sample_binom(n, p):
    k_prob = []
    
    for k in range(0, n+1):
        k_prob.append(binom(n, k, p))

    k_prob_cdf = 0
    U = np.random.rand() # uniform rv.
    for X, k_p in enumerate(k_prob):
        # k_prob_cdf is the value of the binomial CDF at X (so P(X < X))
        k_prob_cdf += k_p
        if U <= k_prob_cdf:
            return X

The mean of a rv $X \sim{} Bin(n,p)$ is $np$ and its variance is $np(1-p)$. So lets check with a simple simulation if our sampling algorithm produces valid rvs. according to $Bin(n,p)$.

In [4]:
n_simulations = 10000

n = 20
p = 0.25
mean = n*p
variance = n*p*(1-p)

x_sum = 0
sim_var = 0
for i in range(n_simulations):
    x = sample_binom(n, p)
    x_sum += x
    sim_var += np.power(mean-x, 2) / n_simulations
    
sim_mean = x_sum / n_simulations

print("Simulated mean:", sim_mean, "mathematical mean:", mean)
print("Simulated variance:", np.around(sim_var,3), "mathematical variance:", variance)

Simulated mean: 5.0037 mathematical mean: 5.0
Simulated variance: 3.715 mathematical variance: 3.75


## Sample standard normal random-variable
Since the inverse of $\Phi(Z)$ is not defined, we will again have to use approximations. Just like students use the z-score table. So for us $\Phi^{-1}(U)=Z$, where $Z$ is a standard normal rv.

First we create the inverse table.

In [310]:
inv_table = {}
X_max = 10.0
X_step = 0.001

X_range = np.linspace(-X_max, X_max, int(1 / X_step))
X_CDF_prob = norm.cdf(X_range)

In [311]:
# Create inverse-phi-table
for i in range(len(X_range)):
    inv_table[X_CDF_prob[i]] = X_range[i]

In [312]:
def sample_normal(mu=0, sigma=1):
    U = np.random.rand()
    
    for i in range(len(X_range)-1):
        if X_CDF_prob[i] < U < X_CDF_prob[i+1]:
            if np.abs(U - X_CDF_prob[i]) < np.abs(U - X_CDF_prob[i+1]):
                return mu + sigma * X_range[i]
            return mu + sigma * X_range[i+1]

In [313]:
sample_normal()

0.6306306306306304

#### Simulate and check if rvs. are valid

In [318]:
n_simulations = 10000

X_mean = 0
X_var = 0
mu = 3
sigma = 4

for i in range(n_simulations):
    z = sample_normal(mu=mu, sigma=sigma)
    X_mean += z
    X_var += np.power(3-z, 2) / n_simulations

X_mean /= n_simulations

In [320]:
print("Simulated mean:", X_mean, "mathematical mean:", mu)
print("Simulated variance:", X_var, "mathematical variance", sigma**2)

Simulated mean: 2.9991991991992006 mathematical mean: 3
Simulated variance: 16.033124135145965 mathematical variance 16


##### Check ✓